In [ ]:
!pip install transformers

다음 주관식 질문에 올바른 답을 작성하시오.

1. bert-base-uncased의 tokenizer를 이용해 "I love natural language processing"을 tokenize한 뒤 token, id, string을 반환하는 코드를 transformers 라이브러리를 이용해 작성하세요.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
print(tokenizer.tokenize("I love natural language processing"))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I love natural language processing")))
print(tokenizer.convert_tokens_to_string(tokenizer.tokenize("I love natural language processing")))

['i', 'love', 'natural', 'language', 'processing']
[1045, 2293, 3019, 2653, 6364]
i love natural language processing


2. transformers의 pipeline class를 이용해 bert-base-uncased 모델을 load하고 "I [MASK] natural language processing"에서 "[MASK]"에 대한 예측값 상위 5개를 출력하는 코드를 작성하세요

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModel

In [ ]:
classifier_ = pipeline("fill-mask")
classifier_("I <mask> natural language processing")

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


[{'score': 0.1383543610572815,
  'sequence': 'Ironic natural language processing',
  'token': 18731,
  'token_str': 'ronic'},
 {'score': 0.12347400188446045,
  'sequence': 'I prefer natural language processing',
  'token': 6573,
  'token_str': ' prefer'},
 {'score': 0.09194808453321457,
  'sequence': 'IEEE natural language processing',
  'token': 47146,
  'token_str': 'EEE'},
 {'score': 0.03860380873084068,
  'sequence': 'I love natural language processing',
  'token': 657,
  'token_str': ' love'},
 {'score': 0.03268039971590042,
  'sequence': 'I am natural language processing',
  'token': 524,
  'token_str': ' am'}]

3. bert-base-uncased의 tokenizer에서 1002번째 token의 string을 출력하는 코드를 작성하세요.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
token = tokenizer.convert_ids_to_tokens(1002)

In [ ]:
print(tokenizer.convert_tokens_to_string(token))

$


4. 다음 출력 문장과 정답 문장을 이용해 BLEU score를 계산하시오. (BLEU score 계산법은 강의자료를 따라주세요.) 

Brevity Score: 0.5  
Predicted Sentence: I like nlp but not you  
Reference: I love natural language processing how about you ?

length of reference = 9  
lenght of prediction = 6  
precision (1-gram) = 2/6  
precision (2-gram) = 0  
precision (3-gram) = 0  
precision (4-gram) = 0  
brevity penalty = 6/9  
BLEU score = 0  

---

다음 명제에 대해, True/False를 판단하시오. 판단 근거를 간략하게 설명하시오.

1. GPT-1,2,3 은 Bi-directional Transformer이다.

False. Transformer의 Decoder를 사용하여 sequence에서 뒤의 token들은 making한다.  

---

2. GPT-1은 여러가지의 task를 하나의 모델로 수행할 수 있다.

True. pretrain 후에 여러 task에 대해 fine-tuning 할 수 있다.  

---

3. BERT는 classification을 하기 위해 encoder(transformer block)의 output을 모두 활용한다.

False. [CLS] token을 사용한다.  

---

4. BERT의 [CLS] token은 반드시 첫번째 sequence에 위치 해야한다.

True. 입력과 출력 모두 맨 앞에 [CLS]token이 위치한다.  

---

5. 두 문장의 의미가 서로 같은지 다른지 판단하는 task를 BERT를 이용하여 수행할 때 문장의 위치가 서로 바뀌더라도 성능은 동일하다.

False. BERT는 Bi-directional Transformer 모델이지만 입력 token들에 position embedding의 정보가 함께 들어가기 때문이다.

---

### build_bpe 함수를 완성해주세요.

byte pair encoding을 이용한 간단한 sub-word tokenizer를 구현해봅니다.
과제 노트북의 지시사항과 각 함수의 docstring과 [논문](https://arxiv.org/pdf/1508.07909.pdf)의 3페이지 algorithm 1 참고하여 build_bpe 함수를 완성하고 모든 test case를 통과해주세요.

In [ ]:
from typing import List, Dict, Set
from itertools import chain
import re
from collections import defaultdict, Counter


def build_bpe(
        corpus: List[str],
        max_vocab_size: int
) -> List[int]:
    """ BPE Vocabulary Builder
    Implement vocabulary builder for byte pair encoding.
    Please sort your idx2word by subword length in descending manner.

    Hint: Counter in collection library would be helpful

    Note: If you convert sentences list to word frequence dictionary,
          building speed is enhanced significantly because duplicated words are
          preprocessed together

    Arguments:
    corpus -- List of words to build vocab
    max_vocab_size -- The maximum size of vocab

    Return:
    idx2word -- Subword list
    """
    # Special tokens
    PAD = BytePairEncoding.PAD_token  # Index of <PAD> must be 0
    UNK = BytePairEncoding.UNK_token  # Index of <UNK> must be 1
    CLS = BytePairEncoding.CLS_token  # Index of <CLS> must be 2
    SEP = BytePairEncoding.SEP_token  # Index of <SEP> must be 3
    MSK = BytePairEncoding.MSK_token  # Index of <MSK> must be 4
    SPECIAL = [PAD, UNK, CLS, SEP, MSK]

    WORD_END = BytePairEncoding.WORD_END  # Use this token as the end of a word

    idx2word: List[str] = SPECIAL
    _corpus = [word + WORD_END for word in corpus]
    vocab = []
    for word in _corpus:
        vocab+=list(word)
    vocab = set(vocab)
    corpus =[]
    for word in _corpus:
        letter = list(word)
        word_ = ' '.join(letter)
        corpus.append(word_)
    corpus = Counter(corpus)

    '''
    Arguments:
        vocab: Counter dict
    Return:
        pairs: dictionary with key as tuple of symbols and value as its count in corpus
    '''

    def get_stats(vocab):
        pairs = defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i + 1])] += freq

        return pairs

    def merge_vocab(pair, v_in):
        v_out = {}
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in v_in:
            w_out = p.sub(''.join(pair), word)
            v_out[w_out] = v_in[word]
        return v_out

    # pair_count_dict: dictionary with key as tuple of symbols and value as its count in corpus
    while(len(vocab)+len(SPECIAL) < max_vocab_size):
        pair_count_dict = get_stats(corpus)
        if len(pair_count_dict) == 0:
            break
        best = max(pair_count_dict, key=pair_count_dict.get)
        vocab.add(best[0]+best[1])
        corpus = merge_vocab(best, corpus)
        
    new_idx2word = list(vocab)
    new_idx2word.sort(reverse=True, key=len)
    idx2word += new_idx2word
    return idx2word

def encode(
    sentence: List[str],
    idx2word: List[str]
) -> List[int]:
    """ BPE encoder
    Implement byte pair encoder which takes a sentence and gives the encoded tokens
    Arguments:
    sentence -- The list of words which need to be encoded.
    idx2word -- The vocab that you have made on the above build_bpe function.
    
    Return:
    tokens -- The list of the encoded tokens
    """
    WORD_END = BytePairEncoding.WORD_END

    def tokenize_word(word, vocab, bias):
        UNK = BytePairEncoding.UNK_token
        # Define base case to stop recursion
        if word == '':
            return []
        if len(vocab) == 0:
            return [UNK]

        # i: index for current vocab
        # bias + i: index for global vocab
        # skip tokens until we find the token that satisfies the pattern
        # word = [substring + token]*n + remainder
        #  => substrings and remainder are fed  recursively to this function with partial vocab
        for i in range(len(vocab)):
            token = vocab[i]
            pattern = re.escape(token)
            matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(pattern, word)]

            if len(matched_positions) == 0:
                continue

            substring_end_positions = [matched_position[0] for matched_position in matched_positions]
            substring_start_position = 0

            # substring start ~ substring end: recursively tokenize with vocabulary of idx2word[i+1:]
            # substring end ~ token[i] length: tokenize into token[i]
            # update substring start to be substring end + len(token[i])
            # iterate until there is no more matched token in the string
            tokens = []
            for substring_end_position in substring_end_positions:
                substring = word[substring_start_position:substring_end_position]
                tokens += tokenize_word(substring, vocab[i + 1:], i + bias + 1)
                tokens.append(i + bias)
                substring_start_position = substring_end_position + len(token)

            # Take care of the remaining substring by recursive call with vocab idex2word[i+1:] then break
            remaining_substring = word[substring_start_position:]
            tokens += tokenize_word(remaining_substring, vocab[i + 1:], i + bias + 1)
            break
        return tokens
    tokens = []
    WORD_END = BytePairEncoding.WORD_END
    for word in sentence:
        word_tokenized = tokenize_word(word + WORD_END, idx2word, 0)
        tokens += word_tokenized

    return tokens

def decode(
    tokens: List[int],
    idx2word: List[str]
) -> List[str]:
    """ BPE decoder
    Implement byte pair decoder which takes tokens and gives the decoded sentence.
    Arguments:
    tokens -- The list of tokens which need to be decoded
    idx2word -- the vocab that you have made on the above build_bpe function.
    Return:
    sentence  -- The list of the decoded words
    """
    WORD_END = BytePairEncoding.WORD_END
    sentence: List[str] = None
    sentence = [idx2word[token] for token in tokens]
    sentence2 = ""
    for i in range(len(sentence)):
        sentence2 += sentence[i]
    sentence = sentence2.split(WORD_END)
    if sentence[-1] == "":
        sentence.pop(-1)
    ### END YOUR CODE
    return sentence

In [ ]:
#############################################
# Helper functions below. DO NOT MODIFY!    #
#############################################

class BytePairEncoding(object):
    """ Byte Pair Encoding class
    We aren't gonna use this class for encoding. Because it is too slow......
    We will use sentence piece Google have made.
    Thus, this class is just for special token index reference.
    """
    PAD_token = '<pad>'
    PAD_token_idx = 0
    UNK_token = '<unk>'
    UNK_token_idx = 1
    CLS_token = '<cls>'
    CLS_token_idx = 2
    SEP_token = '<sep>'
    SEP_token_idx = 3
    MSK_token = '<msk>'
    MSK_token_idx = 4

    WORD_END = '_'

    def __init__(self, corpus: List[List[str]], max_vocab_size: int) -> None:
        self.idx2word = build_bpe(corpus, max_vocab_size)

    def encode(self, sentence: List[str]) -> List[int]:
        return encode(sentence, self.idx2word)

    def decoder(self, tokens: List[int]) -> List[str]:
        return decode(tokens, self.idx2word)


#############################################
# Testing functions below.                  #
#############################################


def test_build_bpe():
    print("======Building BPE Vocab Test Case======")
    PAD = BytePairEncoding.PAD_token
    UNK = BytePairEncoding.UNK_token
    CLS = BytePairEncoding.CLS_token
    SEP = BytePairEncoding.SEP_token
    MSK = BytePairEncoding.MSK_token
    WORD_END = BytePairEncoding.WORD_END

    # First test
    corpus = ['abcde']
    vocab = build_bpe(corpus, max_vocab_size=15)
    assert vocab[:5] == [PAD, UNK, CLS, SEP, MSK], \
        "Please insert the special tokens properly"
    print("The first test passed!")

    # Second test
    assert sorted(vocab[5:], key=len, reverse=True) == vocab[5:], \
        "Please sort your idx2word by subword length in decsending manner."
    print("The second test passed!")

    # Third test
    corpus = ['low'] * 5 + ['lower'] * 2 + ['newest'] * 6 + ['widest'] * 3
    vocab = set(build_bpe(corpus, max_vocab_size=24))
    assert vocab > {PAD, UNK, CLS, SEP, MSK, 'est_', 'low', 'newest_', \
                    'i', 'e', 'n', 't', 'd', 's', 'o', 'l', 'r', 'w',
                    WORD_END} and \
           "low_" not in vocab and "wi" not in vocab and "id" not in vocab, \
        "Your bpe result does not match expected result"
    print("The third test passed!")

    # forth test
    corpus = ['aaaaaaaaaaaa', 'abababab']
    vocab = set(build_bpe(corpus, max_vocab_size=13))
    assert vocab == {PAD, UNK, CLS, SEP, MSK, 'aaaaaaaa', 'aaaa', 'abab', 'aa',
                     'ab', 'a', 'b', WORD_END}, \
        "Your bpe result does not match expected result"
    print("The forth test passed!")

    # fifth test
    corpus = ['abc', 'bcd']
    vocab = build_bpe(corpus, max_vocab_size=10000)
    assert len(vocab) == 15, \
        "Your bpe result does not match expected result"
    print("The fifth test passed!")

    print("All 5 tests passed!")
test_build_bpe()

======Building BPE Vocab Test Case======
The first test passed!
The second test passed!
The third test passed!
The forth test passed!
The fifth test passed!
All 5 tests passed!
